# Simulation

Wind speed -> Power -> H2

In [1]:
import torch
import numpy as np
import pandas as pd
from REStats.utils import load_SCADA, filter_outliers, downsample, standardize
from REStats.models import backtest, power_curve
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
wt_18 = load_SCADA(2018)
wt_19 = load_SCADA(2019)

## Train + Generate Forecast

In [3]:
fcast_train = wt_19[(wt_19.index >= "2019-02-01") & (wt_19.index < "2019-03-01")].copy()
test = wt_19[(wt_19.index >= "2019-03-01") & (wt_19.index < "2019-04-01")].copy()
test = downsample(test)

In [4]:
forecasts_full, (fcast_rmse, fcast_mae) = backtest(downsample(fcast_train), test)

fcast_rmse, fcast_mae

/home/cam/mambaforge/envs/REStats/lib/python3.11/site-packages/pyro/infer/mcmc/api.py:497: UserWarning: num_chains=2 is more than available_cpu=1. Chains will be drawn sequentially.
  warnings.warn(
Sample [0]: 100%|█| 1500/1500 [00:12, 116.60it/s, step size=7.37e-01, acc. prob=
Sample [1]: 100%|█| 1500/1500 [00:13, 109.30it/s, step size=8.59e-01, acc. prob=


Forecast RMSE: 1.114974537660918
Forecast MAE: 0.8302878104288293


(1.114974537660918, 0.8302878104288293)

## Train + Predict Power Curve

In [5]:
pc_train_raw = pd.concat([wt_18, fcast_train])
cut_in = 2.0
cut_out = 18.0
pc_filtered = filter_outliers(pc_train_raw)
pc_train = downsample(pc_filtered)
pc_train = pc_train[(pc_train.wind_speed >= cut_in) & (pc_train.wind_speed < cut_out)]

pc_train.tail()

,wind_speed,wind_dir,power,turbulence_intensity
Date,,,,
2019-02-28 19:00:00,4.730932,286.072572,221.701962,0.035886
2019-02-28 20:00:00,5.325628,300.330975,313.219354,0.030987
2019-02-28 21:00:00,4.284668,308.103162,171.316518,0.037701
2019-02-28 22:00:00,2.904026,316.986263,36.895107,0.060341
2019-02-28 23:00:00,3.489935,316.893271,71.923876,0.042589


In [6]:
kbd = KBinsDiscretizer(n_bins=20, encode="ordinal")
bins = kbd.fit_transform(pc_train.wind_speed.to_numpy()[:, None])

pc_train, _ = train_test_split(pc_train, stratify=bins, train_size=800, random_state=1)

In [7]:
train_std = standardize(pc_train)

X_train = torch.tensor(train_std.wind_speed)
y_train = torch.tensor(train_std.power)

model, likelihood = power_curve.fit(X_train, y_train)

Iter 1/100 - Loss: 0.769   lengthscale: [[0.6931471824645996]]   noise: 0.693
Iter 2/100 - Loss: 0.732   lengthscale: [[0.7443966269493103]]   noise: 0.644
Iter 3/100 - Loss: 0.693   lengthscale: [[0.7979174852371216]]   noise: 0.598
Iter 4/100 - Loss: 0.655   lengthscale: [[0.8535422086715698]]   noise: 0.554
Iter 5/100 - Loss: 0.615   lengthscale: [[0.9110930562019348]]   noise: 0.513
Iter 6/100 - Loss: 0.576   lengthscale: [[0.9703762531280518]]   noise: 0.474
Iter 7/100 - Loss: 0.535   lengthscale: [[1.0311700105667114]]   noise: 0.437
Iter 8/100 - Loss: 0.494   lengthscale: [[1.0932085514068604]]   noise: 0.403
Iter 9/100 - Loss: 0.453   lengthscale: [[1.1561676263809204]]   noise: 0.370
Iter 10/100 - Loss: 0.411   lengthscale: [[1.219658613204956]]   noise: 0.340
Iter 11/100 - Loss: 0.368   lengthscale: [[1.283230185508728]]   noise: 0.312
Iter 12/100 - Loss: 0.325   lengthscale: [[1.3463709354400635]]   noise: 0.286
Iter 13/100 - Loss: 0.282   lengthscale: [[1.4085071086883545]]

In [8]:
test_sort = test.sort_values("wind_speed")
test_sort_std = standardize(test_sort, ref_df=pc_train)
X_test_torch = torch.tensor(test_sort_std.wind_speed)

test_pred = power_curve.predict(model, likelihood, X_test_torch)

In [9]:
test_pred_tf = test_pred.mean.numpy() * pc_train.power.std() + pc_train.power.mean()
test_rmse = mean_squared_error(test_pred_tf, test_sort.power, squared=False)

print(f"Test RMSE: {test_rmse} kW")

Test RMSE: 81.63548640845919 kW


In [10]:
fcast = forecasts_full["mean"]
fcast_std = (fcast - pc_train.wind_speed.mean())/pc_train.wind_speed.std()
fcast_torch = torch.tensor(fcast_std)

test_pred = power_curve.predict(model, likelihood, fcast_torch)

test_pred_tf = test_pred.mean.numpy() * pc_train.power.std() + pc_train.power.mean()

# filter out values outside operating range
pred_filtered = np.zeros(len(test_pred_tf))

for i, pred in enumerate(fcast):
    if (pred < cut_in) or (pred >= cut_out):
        pred_filtered[i] = 0.0
    else:
        pred_filtered[i] = test_pred_tf[i]


test_rmse = mean_squared_error(pred_filtered, test[1:].power, squared=False)
test_mae = mean_absolute_error(pred_filtered, test[1:].power)
print(f"Test RMSE: {test_rmse} kW")
print(f"Test MAE: {test_mae} kW")

Test RMSE: 252.55331885863177 kW
Test MAE: 164.00692680565191 kW
